<a href="https://colab.research.google.com/github/ssvadla/Bridge_Project/blob/main/Supervised_Light_XG_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTrain.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['Unnamed: 0', 'Skew', 'Number_Spans', 'Length_Max_Span',
       'Main_Design_Type', 'Deck_Design_Type', 'Wearing_Surface',
       'Deck_Protection', 'ADT', 'ADTT', 'State', 'Structure_Number',
       'Latitude', 'Longitude', 'Deck_Condition', 'Superstructure_Condition',
       'Substructure_Condition', 'Days_MinLT_0F', 'Days_MaxGT_90F',
       'Avg_Wind_Speed', 'Peak_Wind_Speed', 'Precipitation', 'Reconstructed',
       'Age', 'Deck_Area'],
      dtype='object')

In [4]:
#Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude', 'Superstructure_Condition', 'Substructure_Condition']
Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude']

In [5]:
train = train.drop(columns = Excluded_columns)

In [6]:
train.shape

(41902, 20)

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Main_Design_Type_Encoded'] = le.fit_transform(train['Main_Design_Type'])
train['Deck_Design_Type_Encoded'] = le.fit_transform(train['Deck_Design_Type'])
train['Wearing_Surface_Encoded'] = le.fit_transform(train['Wearing_Surface'])
train['Deck_Protection_Encoded'] = le.fit_transform(train['Deck_Protection'])


In [8]:
 for value in train['Substructure_Condition']:
   #print(value)
   if value == 'N':
     print("Yes")

Yes


In [9]:
indexNames = train[ train['Substructure_Condition'] == 'N' ].index

In [10]:
indexNames

Int64Index([35702], dtype='int64')

In [11]:
train.shape

(41902, 24)

In [12]:
train.drop(indexNames , inplace=True)

In [13]:
train.shape

(41901, 24)

In [14]:
train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])

In [15]:
np.unique(train['Substructure_Condition_typed'].values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
train['Substructure_Condition_Encoded'] = le.fit_transform(train['Substructure_Condition_typed'])

In [17]:
import lightgbm as lgb
classifier_lgb = lgb.LGBMClassifier()

In [18]:
X = train.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition_typed','Substructure_Condition']) 


In [19]:
y = train['Deck_Condition']

In [20]:
X.shape

(41901, 19)

In [21]:
start = time.time()
classifier_lgb.fit(X, y)
end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 3.0992355346679688


In [22]:
test = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTest.csv')

In [23]:
test = test.drop(columns = Excluded_columns)

In [24]:
test['Main_Design_Type_Encoded'] = le.fit_transform(test['Main_Design_Type'])
test['Deck_Design_Type_Encoded'] = le.fit_transform(test['Deck_Design_Type'])
test['Wearing_Surface_Encoded'] = le.fit_transform(test['Wearing_Surface'])
test['Deck_Protection_Encoded'] = le.fit_transform(test['Deck_Protection'])

In [25]:
#train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])
test['Substructure_Condition_Encoded'] = le.fit_transform(test['Substructure_Condition'])

In [26]:
X_test = test.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition']) 


In [27]:
y_test = test['Deck_Condition']

In [28]:
np.unique(train['Deck_Condition'].values)

array([ 6,  7, 45, 89])

In [29]:
np.unique(test['Deck_Condition'].values)

array([ 6,  7, 45, 89])

In [30]:
X_test.shape

(13970, 19)

In [31]:
test_pred = classifier_lgb.predict(X_test)

In [32]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test,test_pred, digits=4)

In [33]:
print(classification_report)

              precision    recall  f1-score   support

           6     0.6480    0.6079    0.6273      3861
           7     0.6802    0.7348    0.7065      4453
          45     0.7186    0.7010    0.7097      2746
          89     0.8178    0.8034    0.8105      2910

    accuracy                         0.7074     13970
   macro avg     0.7161    0.7118    0.7135     13970
weighted avg     0.7075    0.7074    0.7069     13970



In [34]:
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
classifier_xg = XGBClassifier()
start = time.time()
classifier_xg.fit(X, y)
end = time.time()
test_pred_xg = classifier_xg.predict(X_test)
classification_report_xg = classification_report(y_test,test_pred_xg, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_xg)

Runtime of the program is 12.650245666503906
              precision    recall  f1-score   support

           6     0.6463    0.5789    0.6107      3861
           7     0.6522    0.7357    0.6914      4453
          45     0.7080    0.6712    0.6891      2746
          89     0.8053    0.7986    0.8019      2910

    accuracy                         0.6928     13970
   macro avg     0.7030    0.6961    0.6983     13970
weighted avg     0.6934    0.6928    0.6917     13970



In [35]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
start = time.time()
classifier_rf.fit(X, y)
end = time.time()
test_pred_rf = classifier_rf.predict(X_test)
classification_report_rf = classification_report(y_test,test_pred_rf, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_rf)

Runtime of the program is 30.163610458374023
              precision    recall  f1-score   support

           6     0.6595    0.6291    0.6440      3861
           7     0.6944    0.7411    0.7170      4453
          45     0.7373    0.7163    0.7266      2746
          89     0.8260    0.8137    0.8198      2910

    accuracy                         0.7204     13970
   macro avg     0.7293    0.7251    0.7268     13970
weighted avg     0.7206    0.7204    0.7201     13970



In [36]:
from sklearn import svm
classifier_svm = svm.SVC(decision_function_shape='ovo')
start = time.time()
classifier_svm.fit(X, y)
end = time.time()
test_pred_svm = classifier_svm.predict(X_test)
classification_report_svm = classification_report(y_test,test_pred_svm, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_svm)

Runtime of the program is 147.14262795448303
              precision    recall  f1-score   support

           6     0.3712    0.0862    0.1400      3861
           7     0.3187    0.9347    0.4753      4453
          45     0.4615    0.0022    0.0043      2746
          89     0.0000    0.0000    0.0000      2910

    accuracy                         0.3222     13970
   macro avg     0.2879    0.2558    0.1549     13970
weighted avg     0.2949    0.3222    0.1910     13970



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
